<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Одномерное интегрирование методом Монте-Карло #

### Modules - Numerical Integration
<section class="post-meta">
By Tor Nordam
</section>
___
This notebook will give a brief introduction to one dimensional numerical integration, comparing two naïve methods:
  * Riemann Sum
  * Monte Carlo integration with uniform sampling
 

## Сумма Римана ##
Сумма Римана, пожалуй, является самой простой и интуитивно понятной схемой численного интегрирования. Вы хотите интегрировать функцию на интервале длины $x$. Разделите свой интервал на $N$ подинтервалов равной длины $\Delta x = L/N$. Оцените функцию в середине каждого подинтервала. Вклад в общую площадь от субинтевала со средней точкой $x_i$ тогда равен $f(x_i) \Delta x$, а общее значение интеграла равно
$$
\Delta x\sum_{i=1}^{N} f(x_i).
$$

Следующий блок кода создает график, который визуализирует процедуру римановой суммы

In [ ]:
# Эти строки импортируют библиотеку numpy, настраивают matplotlib для использования 
# непосредственно в ноутбуке и заставляют деление работать так же, как в Python 3, 
# так что 1/2 = 0.5 вместо 1/2 = 0

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
#from __future__ import division


# Установит общие параметры фигуры:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': True, 
             'legend.handlelength': 1.5}
plt.rcParams.update(newparams)


# Определит функцию для использования в качестве примера
def f(x):
    return x**2

# Очистит участок и установит ограничения
plt.clf()
start = 0
stop  = 1
plt.xlim(start, stop)
plt.ylim(start, stop)

# Построbn график интегрируемой функции, используя 1000 точек
# чтобы получить плавную кривую
X = np.linspace(start, stop, 1000)
plt.plot(X, f(X))

# Нарисует прямоугольники, используемые в сумме Римана
# а именно, N прямоугольников
N  = 10
# Каждый из них имеет ширину dx
dx = (stop - start)/N
# Создаст вектор средних точек каждого прямоугольника
X  = np.linspace(start + dx/2, stop - dx/2, N)
# Нарисует средние точки в виде кругов
plt.scatter(X, f(X))
# Для каждого прямоугольника нарисует границы с помощью линейного графика
for x in X:
    plt.plot([x-dx/2, x-dx/2, x+dx/2, x+dx/2], [0,f(x), f(x), 0], color = "g")

## Интегрирование методом Монте-Карло с равномерной выборкой ##

Простейшая схема интегрирования Монте-Карло по существу очень похожа на метод суммы Римана, за исключением того, что точки, в которых должна оцениваться функция, выбираются случайным образом, а не на равном расстоянии друг от друга. Равномерная выборка означает, что каждая точка имеет одинаковую вероятность выбора. Более продвинутые версии Монте-Карло могут, например, использовать более плотную выборку в областях, где значение функции быстро меняется (*т.е.*, где производная функции велика).

Для большого числа случайных точек интеграл, полученный методом Монте-Карло, приблизится к истинному значению интеграла.

Следующий блок кода создает график, который визуализирует этот метод.

In [ ]:
# Определит функцию для использования в качестве примера
def f(x):
    return x**2

# Очистит рисунок и установит ограничения
plt.clf()
start = 0
stop  = 1
plt.xlim(start, stop)
plt.ylim(start, stop)

# Построит график интегрируемой функции, используя 1000 точек
# чтобы получить плавную кривую
X = np.linspace(start, stop, 1000)
plt.plot(X, f(X))

# Нарисует прямоугольники, используемые в методе Монте-Карло
# Есть N прямоугольников
N  = 10
# Каждый из них имеет ширину dx
dx = (stop - start)/N
# Создает вектор средних точек каждого прямоугольника,
# используя однородные случайные числа из функции np.random.random()
# Эта функция возвращает числа на интервале [0, 1).
# Мы масштабируем числа до интервала [a, b) путем умножения
# на длину интервала и добавлением начальной точки
# Вы можете легко удостовериться, что это работает
X  = np.random.random(N)*(stop - start) + start
# Нарисует средние точки в виде кругов
plt.scatter(X, f(X))
# Для каждого прямоугольника нарисует границы с помощью линейного графика
for x in X:
    plt.plot([x-dx/2, x-dx/2, x+dx/2, x+dx/2], [0,f(x), f(x), 0], color = "g")

## Определение функций для численного интегрирования ##
Далее мы определим две функции для проведения численного интегрирования с использованием суммы Римана и метода Монте-Карло с равномерной выборкой. Мы будем использовать тот факт, что в Python можно отправить функцию в качестве аргумента другой функции, чтобы сделать общие интеграторы, которые можно использовать для любой функции одной переменной.

### Сумма Римана ###

In [ ]:
# Аргументами этой функции являются:
# f, функция, которая должна быть интегрирована. Должен принимать только один аргумент
# N, количество точек для оценки функции
# start, начало интервала интегрирования
# stop, конец интервала интегрирования
def riemannSum(f, N, start, stop):
    # Ширина каждого прямоугольника
    dx = (stop - start) / N
    # Массив с серединой каждого прямоугольника
    X = np.linspace(start + dx/2, stop - dx/2, N)
    # Когда f(x) применяется к массиву, он возвращает массив 
    # соответствующего размера, удерживая результат применения функции
    # к каждому элементу исходного массива. Берем сумму 
    # результирующего массива и умножить на dx
    return sum(f(X))*dx

### Монте-Карло ###

In [ ]:
# Аргументами этой функции являются:
# f, функция, которая должна быть интегрирована. Должен принимать только один аргумент
# N, количество точек для оценки функции
# start, начало интервала интегрирования
# stop, конец интервала интегрирования
def monteCarloIntegration(f, N, start, stop):
    # Ширина каждого прямоугольника
    dx = (stop - start) / N
    # Массив с серединой каждого прямоугольника, из однородных случайных чисел
    R = np.random.random(N)*(stop - start) + start
    # Когда f(x) применяется к массиву, он возвращает массив 
    # соответствующего размера, удерживая результат применения функции
    # к каждому элементу исходного массива. Берем сумму 
    # результирующего массива и умножить на dx
    return sum(f(R))*dx

## Приложение 1 ##
Здесь мы применим наши две различные схемы интегрирования для вычисления простого интеграла
$$ \int_0^1 x^2 \;\mathrm{d}x = \frac{1}{3} $$
и сравним результаты. Обратите внимание, что сумма Римана обычно работает лучше для этого примера, *т.е.*, она дает лучшую точность для одного и того же числа оценок функции (функция оценивается $N$ раз в обоих случаях). Обратите также внимание, что сумма Римана всегда дает один и тот же результат, в то время как метод Монте-Карло будет колебаться случайным образом. Обратите внимание, как точность обоих увеличивается, когда вы увеличиваете количество точек, $N$.

In [ ]:
# Простой пример функции
def f(x):
    return x**2

# Определит количество точек и интервал
N = 40
start = 0
stop  = 1
print("Сумма Римана: ", riemannSum(f, N, start, stop) )
print("Монте-Карло: ", monteCarloIntegration(f, N, start, stop) )

## Приложение 2 ##
Почему мы утруждаем себя разговорами о методе Монте-Карло, когда сумма Римана, кажется, работает лучше? Потому что это не всегда так. Здесь мы рассмотрим другой пример, интеграл
$$ \int_0^1 \sin^2 \left( 20 \cdot 2\pi x \right)\; \mathrm{d}x = \frac{1}{2}.$$

In [ ]:
# Простой пример функции
def f(x):
    return np.sin(20*2*np.pi*x)**2

# Определит количество точек и интервал
N = 40
start = 0
stop  = 1
print("Сумма Римана: ", riemannSum(f, N, start, stop) )
print("Монте-Карло: ", monteCarloIntegration(f, N, start, stop) )

Так что же здесь произошло? Результат римановой суммы равен 1.0, в то время как результат интегрирования по методу Монте-Карло находится, по крайней мере, в правильной окрестности. Причина в том, что функция, которую мы рассматривали, $f(x) = \cos^2 ( 20\cdot 2 \pi x)$, варьируется от 0 до 1 и имеет 40 локальных максимумов на интервале от 0 до 1. Используя сумму Римана, мы произвели выборку в 40 точках, и, как это бывает, эти 40 точек в точности соответствуют 40 максимумам. Обратите внимание, что изменение количества точек (даже до 41) дает лучший ответ для метода Римана.

Это, конечно, пример, который построен так, чтобы метод суммы Римана работал плохо, но принцип остается в силе. Для быстро осциллирующих функций интегратор будет чувствителен к интервалу интегрирования. В этих случаях интегрирование по методу Монте-Карло может работать лучше, чем схемы, использующие одинаково распределенные оценки функций.

## Бонус ##

Вот два блока кода, которые визуализируют приведенный выше пример.

In [ ]:
# Функция, которая должна быть интегрирована
def f(x):
    return np.sin(20*2*np.pi*x)**2

# Очистит график и установит ограничения и размер фигуры
# Мы строим интервал от 0 до 0.4 только для того, чтобы его было легче увидеть
plt.clf()
plt.xlim(0, 0.4)
plt.ylim(0, 1.1)

# Строит график интегрируемой функции, используя 1000 точек
# чтобы получить плавную кривую
start = 0
stop  = 1
X = np.linspace(start, stop, 1000)
plt.plot(X, f(X))

# Рисует прямоугольники, используемые в сумме Римана
N  = 40
dx = (stop - start)/N
X  = np.linspace(start + dx/2, stop - dx/2, N)
# Рисует средние точки в виде кругов
plt.scatter(X, f(X))
# Для каждого прямоугольника нарисует границы с помощью линейного графика
for x in X:
    plt.plot([x-dx/2, x-dx/2, x+dx/2, x+dx/2], [0,f(x), f(x), 0], color = "g")

In [ ]:
# Функция, которая должна быть интегрирована
def f(x):
    return np.sin(20*2*np.pi*x)**2

# Очистит график и установит ограничения и размер фигуры
# Мы строим интервал от 0 до 0,4 только для того, чтобы его было легче увидеть
plt.clf()
plt.xlim(0, 0.4)
plt.ylim(0, 1.1)

# Построит график интегрируемой функции, используя 1000 точек
# чтобы получить плавную кривую
start = 0
stop  = 1
X = np.linspace(start, stop, 1000)
plt.plot(X, f(X))

# Нарисует прямоугольники, используемые в сумме Римана
N  = 40
dx = (stop - start)/N
X  = np.random.random(N)*(stop - start) +  start
# Нарисует средние точки в виде кругов
plt.scatter(X, f(X))
# Для каждого прямоугольника нарисует границы с помощью линейного графика
for x in X:
    plt.plot([x-dx/2, x-dx/2, x+dx/2, x+dx/2], [0,f(x), f(x), 0], color = "g")